In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import re
import requests
import random
import seaborn as sns

def getYears(df):
    return df["Year"].unique()

def create_index(df, idx):
    total_events = df.shape[0]
    df[idx] = (
        df["Year"].astype(str)
        + "_"
        + df["Month"].astype(str).str.zfill(2)
        + "_"
        + df["latitude"].round(3).astype(str)
        + "_"
        + df["longitude"].round(3).astype(str)
        + "_"
        + [str(uuid.uuid4())[:8] for _ in range(total_events)]
    )
    return df

def classify_earthquake(row):
    try:
        mag = float(row["magnitude"])
        depth = float(row["depth"])
        gap = float(row["gap"])
        tsunami = int(row["tsunami"])
    except:
        return "Light"  # si hay error en datos, lo marcamos como leve

    # Light → soft or depth seismics
    if (mag < 5.2) or (mag < 5 and depth > 150):
        return "Light"

    # Medium → moderate seismics
    elif (5.2 <= mag < 6.8 and depth <= 200 and tsunami == 0):
        return "Medium"

    # Tough → high magnitude or tsunami
    elif (mag >= 6.8) or (depth < 50) or (tsunami == 1):
        return "Tough"

    else:
        return "Light"  # Ensure that returning "Light"

def display_values(ax):
    total = sum([p.get_height() for p in ax.patches])
    for patch in ax.patches:
        height = patch.get_height()
        percentage = (height / total) * 100 if total > 0 else 0
        ax.text(
            patch.get_x() + patch.get_width() / 2,
            height + 1,
            f"{int(height)} ({percentage:.1f}%)",
            ha="center", va="bottom", fontsize=10
        )
    return ax

def full_info():
    return print("""#------ INFO ------#
# | Column        | Meaning                          | Unit / Scale                                                | Description                                                                                             |
# | ------------- | -------------------------------- | ----------------------------------------------------------- | ------------------------------------------------------------------------------------------------------- |
# | **magnitude** | Earthquake magnitude             | Richter or Moment Magnitude (Mw)                            | A measure of the energy released. Higher = stronger quake.                                              |
# | **cdi**       | *Community Determined Intensity* | I–X scale (Modified Mercalli Intensity, average from users) | Based on public “Did You Feel It?” reports. Reflects **felt intensity**.                                |
# | **mmi**       | *Modified Mercalli Intensity*    | I–X scale                                                   | Estimated shaking intensity from instruments, not people.                                               |
# | **sig**       | *Significance*                   | Numeric (0–1000)                                            | Composite score used by USGS to rank event importance (depends on magnitude, felt area, reports, etc.). |
# | **nst**       | *Number of Stations*             | Count                                                       | Number of seismic stations that detected and reported the event. More = higher accuracy.                |
# | **dmin**      | *Minimum Distance*               | Degrees (~111 km per degree)                                | Distance from epicenter to the nearest seismic station. Smaller = better accuracy.                      |
# | **gap**       | *Azimuthal Gap*                  | Degrees                                                     | Largest angle between stations around the epicenter. Smaller = better network coverage.                 |
# | **depth**     | *Hypocenter Depth*               | Kilometers (km)                                             | How deep below the Earth’s surface the earthquake occurred.                                             |
# | **latitude**  | Geographic coordinate            | Degrees                                                     | Latitude of the earthquake’s epicenter.                                                                 |
# | **longitude** | Geographic coordinate            | Degrees                                                     | Longitude of the earthquake’s epicenter.                                                                |
# | **Year**      | Year of occurrence               | Year (e.g. 2024)                                            | Extracted from the `time` field (or event timestamp).                                                   |
# | **Month**     | Month of occurrence              | Month (1–12)                                                | Useful for temporal trend analysis.                                                                     |
# | **tsunami**   | Tsunami alert flag               | 0 = No, 1 = Yes                                             | Whether the earthquake generated a tsunami.                                                             |

# 💡 Example Interpretations
# Example	Meaning
# magnitude = 6.2	Strong earthquake
# cdi = 7, mmi = 6.5	Very strong shaking, felt widely
# sig = 850	Significant event, possibly newsworthy
# nst = 130, dmin = 0.05, gap = 40	Very well-detected and precisely located
# depth = 12.0	Shallow earthquake (more damaging potential)
# tsunami = 1	Tsunami generated — typically near coastlines""")

def load_file(path: str):
    """
    Load a dataset from a local path or URL (including Google Drive).
    Supports CSV, Excel, JSON, Parquet, Feather, and Pickle formats.
    """
    # Map extensions to pandas functions
    extensions = {
        "csv": pd.read_csv,
        "xlsx": pd.read_excel,
        "xls": pd.read_excel,
        "json": pd.read_json,
        "parquet": pd.read_parquet,
        "feather": pd.read_feather,
        "pickle": pd.read_pickle,
    }

    url_pattern = r"^https?:\/\/"

    # If it's a URL
    if re.match(url_pattern, path):
        # Handle Google Drive links
        if "drive.google.com" in path:
            file_id = re.search(r"/d/([a-zA-Z0-9_-]+)", path)
            if file_id:
                path = f"https://drive.google.com/uc?id={file_id.group(1)}"

        # Download content
        response = requests.get(path)
        if response.status_code != 200:
            raise ValueError(f"Error {response.status_code} while accessing URL: {path}")

        # Guess file extension
        extension_match = re.search(r"\.(\w+)(?:\?|$)", path)
        extension = extension_match.group(1).lower() if extension_match else "csv"

        # Read file based on extension
        if extension in ["csv", "txt"]:
            df = pd.read_csv(io.StringIO(response.content.decode("utf-8")))
        elif extension in ["xlsx", "xls"]:
            df = pd.read_excel(io.BytesIO(response.content))
        elif extension == "json":
            df = pd.read_json(io.StringIO(response.content.decode("utf-8")))
        else:
            raise ValueError(f"Unsupported extension in URL: '{extension}'")

    # If it's a local file
    else:
        extension = path.split(".")[-1].lower()
        if extension not in extensions:
            raise ValueError(f"Unsupported file type: {extension}")
        df = extensions[extension](path)

    return df


def replaceAllEmptyByNA(df: pd.DataFrame) -> pd.DataFrame:
    """
    Replace all types of empty/invalid values in the entire DataFrame with pd.NA.
    """
    # Convert columns to best dtype
    df = df.convert_dtypes()

    # Trim spaces in string columns
    df = df.apply(lambda col: col.str.strip() if col.dtype == "string" else col)

    # Replace common "empty" values
    df = df.replace(r"^\s*$", pd.NA, regex=True)  # "", " ", "   "
    df = df.replace({"None": pd.NA, "nan": pd.NA, "NaN": pd.NA})

    return df


def get_mode(col: pd.Series, as_string: bool = False):
    """
    Returns the mode of a column safely.
    - Ignores NA
    - Returns first if multiple
    - Returns None if empty
    """
    col = col.dropna()
    if col.empty:
        return None

    moda = col.mode()
    if moda.empty:
        return None

    val = moda.iloc[0]
    return str(val) if as_string else val


def count_empty_values(col: pd.Series) -> int:
    """Count how many missing values (pd.NA / NaN) are in a column."""
    return col.isna().sum()


def checkIfEmpty(df: pd.DataFrame) -> None:
    """Print a summary if the DataFrame is empty or not."""
    if df.empty:
        print("⚠️ The DataFrame is empty.")
    else:
        print(f"✅ Data loaded successfully: {len(df)} rows, {len(df.columns)} columns.")


def countWhiteSpacesRows(field: pd.Series) -> int:
    """Count how many cells contain whitespace characters."""
    return field.apply(lambda x: any(ch.isspace() for ch in str(x)) if isinstance(x, str) else False).sum()

## Checking and data clean 😎🔎
- Load file
- Check null values
- Have as a guiance a full map info
- Ensure that replacing all empty data by NA to avoid errors after using this following functions: fillna, dropna, isna

In [ ]:
# Load dataset
df = load_file("earthquake_data_tsunami.csv")

In [ ]:
# Check null values
count_empty_values(df)

In [ ]:
full_info()

In [ ]:
df = replaceAllEmptyByNA(df)

df.nunique()

df.duplicated().sum()

### Management and calculus ⚖️ 📶

#### Management 📃

Manage data and do calculus once previous stage is done. This is all about knowing behaviour and get used to it and what's more get on with data well.

- To begin with, as there is not a column which makes each data as unique, we will create a index combined with this following fields
  - Year
  - Month
  - latitude
  - longitude

Rarely, got to see something like this. It is more normal to create an index using timestamp. In this case, there is not a timestamp field so we decide to use this four fields. By far, they are closest to be each other unique. To grow into that, the random is the solution. It would turn out just like this: 

  - Field named "id_event" created by create_index function

- Later on, as we need to handle evolution and temporaly calculus, we got to create a new column named event_date which contains Year and Month column parsed into datetime to use it as index.It will ensure that working resample well. 
- To end with, set global variables "total_events" and "years" which they will be useful for the workflow
 
#### Basic and advanced calculus 🤓

- Basic statistics about magnitude and depth columns
- Top 10 earthquakes by magnitude
- Calculate overall mean and standard deviation
- Show earthquakes closest to the average (most typical magnitudes)
- Show the most dispersed earthquakes (further from the average)
- Detect and display real outliers with the interquartile range (IQR) rule


In [ ]:
# There is not a unique id so we create a id
# First we parse month and year to string and latitude and longitude too
import random

total_events = df.shape[0]

import uuid

# Total number of events
total_events = df.shape[0]

# Create unique ID by combining information and a short hash
df = create_index(df, "id_event")
# Generate date column combined by month and year

df["event_date"] = df["Year"].astype(str)+"/"+df["Month"].astype(str)

df["event_date"] = pd.to_datetime(df["event_date"], format="%Y/%m")

df = df.set_index("event_date")

years = getYears(df)

In [ ]:
# Basic statistics about magnitude and depth columns
stats_data = {
    'Metric': [
        'Mean Magnitude',
        'Median Magnitude',
        'Mode Magnitude',
        'Std Dev (Depth)',
        'Correlation Magnitude-Depth'
    ],
    'Value': [
        df["magnitude"].mean(),
        df["magnitude"].median(),
        get_mode(df["magnitude"]),
        df["depth"].std(),
        df["magnitude"].corr(df["depth"])
    ]
}

stats_df = pd.DataFrame(stats_data)
display(stats_df)

In [ ]:
# Top 10 earthquakes by magnitude
top10 = df.sort_values("magnitude", ascending=False).head(10)
display(top10[["magnitude","depth","gap","sig","tsunami"]])

In [ ]:
# Calculate overall mean and standard deviation
mean_mag = df["magnitude"].mean()
std_mag = df["magnitude"].std()

data_set = {
    'Overall Mean Magnitude': mean_mag,
    'Overall Std Dev Magnitude': std_mag
}

display(pd.DataFrame(data_set, index=[0]))

group_stats = df.groupby("tsunami")["magnitude"].agg(["mean","std","min","max","median"])
display(group_stats)

In [ ]:
# Show earthquakes closest to the average (most typical magnitudes)
df["dist_from_mean"] = abs(df["magnitude"] - mean_mag)
closest_to_mean = df.nsmallest(10, "dist_from_mean")[["magnitude","depth","gap","sig","tsunami"]]
display("Earthquakes closest to the global average:")
display(closest_to_mean)

In [ ]:
# Show the most dispersed earthquakes (further from the average)
farthest_from_mean = df.nlargest(10, "dist_from_mean")[["magnitude","depth","gap","sig","tsunami"]]
display("Earthquakes farthest from the average (most atypical):")
display(farthest_from_mean)

In [ ]:
# Detect and display real outliers with the interquartile range (IQR) rule
Q1 = df["magnitude"].quantile(0.25)
Q3 = df["magnitude"].quantile(0.75)
IQR = Q3 - Q1

# Allowed range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df["magnitude"] < lower_bound) | (df["magnitude"] > upper_bound)]

display(f"Allowed range: {lower_bound:.2f} - {upper_bound:.2f}")
display(f"Outliers detected: {outliers.shape[0]}")

display(outliers[["magnitude","depth","gap","sig","tsunami"]])

### Visualization 🎯 📈

Definitely, this is the end part of dataflow. Let's represent all both implicit and explicit data through different kind of calculus.

- Monthly evolution of magnitude: At first, it orders whatever year by input provided that it exists in dataset. Afterwards, it builds a dataframe filtered by year. To end with, it represents a basic plot displaying the evolution by month, but only for the year filtered. 

- Annual Magnitude Evolution: it represents a basic plot displaying the evolution of magnitudes by year

- Magnitude mean by month: it represents a basic plot displaying the mean of magnitudes by month. At first, it orders whatever year by input provided that it exists in dataset. Afterwards, it builds a dataframe filtered by year. To end with respect to the representation. 

- Earthquake Classification (Magnitude + Depth + Gap + Tsunami): it represents all earthquakes over years. They are classificated not only by magnitude, but also by depth, gap and tsunami. It is worth bearing in mind that rating this kind of classifications because we know by far what the earthqueake most destroyes and keep count by next time we will have known or will know which will be the next earthquake either light, medium or tough through predictions or depth learning algorithms. 

- Distribution of Earthquakes by Year: it represents by percentage the distribution of earthqueakes over years through a pie chart.

- Boxplot dispersion magnitude, depth and gap: it represents the dispersion and how set of data is dispersed over average through boxplot both magnitude, depth and gap over years. What's more it represents ooutliers which they are inusual. 

- Magnitude by Tsunami Presence: it is as same as the previous one but only by tsunami. It represents the dispersion and how set of data is dispersed over average through boxplot either no or yes tsunami and what's more it displays the outlier over years.

- Distribution of magnitude according either not or yes thsunami presence: it represents the distribution f magnitude over years either not or yes tsunami presence. In this case, the reresentation is made by linear chart instead targeting a vertial line intersecting chart which it displays the mean.  

- Correlation Matrix between Earthquake Variables: it represents all variables which one most near to average, disperse or outlier each other from earthquakes through heatmap. 

In [ ]:
# Monthly evolution of magnitude
try:

    year = int(input(f"Type years {",".join(map(str, years ))}"))

    if year not in years:
        raise Exception("Not found year in dataset")

    df_magnitude_year = df[df["Year"] == year]

    monthly_mag_count = df_magnitude_year["magnitude"].resample("ME").count()

    monthly_mag_count.plot(
        figsize=(10,5),
        color="orange",
        title=f"Monthly magnitude count evolution of {year}",
        ylabel="Mean magnitude",
        xlabel="Month",
    )
    plt.show()
except Exception as e:
    display(e)
except NameError as e:
    print(e)
except ValueError as e:
    print(e)
except KeyboardInterrupt as e:
    print(e)

In [ ]:
# Annual evolution of magnitude
annual_counts = df["magnitude"].resample("YE").count()

annual_counts.index = annual_counts.index.year # Access to year and only choose one
annual_counts = annual_counts.sort_index() # sort index ascending

annual_counts.plot(
    kind="bar",
    figsize=(10,5),
    title="Annual evolution of magnitude events",
)

plt.xlabel("Year")
plt.ylabel("Earthquakes number")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Magnitude mean by month
try:

    year = int(input(f"Type years {",".join(map(str, years ))}"))

    if year not in years:
        raise Exception("Not found year in dataset")

    df_magnitude_year = df[df["Year"] == year]

    monthly_mag_mean = df_magnitude_year["magnitude"].resample("ME").mean()

    monthly_mag_mean.plot(
        figsize=(10,5),
        color="orange",
        title=f"Monthly magnitude mean of {year}",
        ylabel="Mean magnitude",
        xlabel="Month",
    )
    plt.show()
except Exception as e:
    display(e)
except NameError as e:
    print(e)
except ValueError as e:
    print(e)
except KeyboardInterrupt as e:
    print(e)

In [ ]:
# Earthquake Classification (Magnitude + Depth + Gap + Tsunami)

df["magnitude"] = pd.to_numeric(df["magnitude"], errors="coerce")
df["depth"] = pd.to_numeric(df["depth"], errors="coerce")
df["gap"] = pd.to_numeric(df["gap"], errors="coerce")
df["tsunami"] = pd.to_numeric(df["tsunami"], errors="coerce")

df["rating_advanced"] = df.apply(classify_earthquake, axis=1)

display(df["magnitude"].min())

# Create a bar plot of earthquake ratings
rating_order = ["Light", "Medium", "Tough"]
rating_counts = df["rating_advanced"].value_counts().reindex(rating_order)

display(count_empty_values(df))

# Create a bar plot with customized style
plt.figure(figsize=(10, 6))
ax = rating_counts.plot(kind="bar", color=["lightblue", "orange", "red"])

plt.title("Earthquake Classification (Magnitude + Depth + Gap + Tsunami)")
plt.xlabel("Rating")
plt.ylabel("Number of Earthquakes")
plt.xticks(rotation=45)

# ✅ Add values on top of bars (with %)
ax = display_values(ax)

plt.tight_layout()
plt.show()

In [ ]:
# Distribution of Earthquakes by Year
total_earthquakes = len(df)
earthquakes_by_year = df["magnitude"].resample("YE").count()
percentages = (earthquakes_by_year / total_earthquakes) * 100

labels = [f"{int(year)} - {count} ({p:.1f}%)" for year, count, p in zip(
    earthquakes_by_year.index.year, earthquakes_by_year, percentages)]

plt.figure(figsize=(8,8))
plt.pie(earthquakes_by_year, labels=labels, startangle=90, autopct="%1.1f%%")
plt.title("Distribution of Earthquakes by Year")
plt.show()


In [ ]:
# Dispersion and how set of data is dispersed over average through boxplot both magnitude, depth and gap over years
fig, axes = plt.subplots(1, 3, figsize=(15,5))
display("Boxplot dispersion magnitude, depth and gap")
sns.boxplot(y=df["magnitude"], ax=axes[0], color="skyblue")
sns.boxplot(y=df["depth"], ax=axes[1], color="lightcoral")
sns.boxplot(y=df["gap"], ax=axes[2], color="violet")
axes[0].set_title("Magnitude"); axes[1].set_title("Depth"); axes[2].set_title("Gap")
plt.show()

In [ ]:
# Dispersion and how set of data is dispersed over average through boxplot either no or yes tsunami and what's more it displays the outlier over years
fig, ax = plt.subplots(figsize=(6,5))
colors = ["lightgray" if t==0 else "red" for t in sorted(df["tsunami"].unique())]
sns.boxplot(data=df, x="tsunami", y="magnitude", hue="tsunami", legend=False, palette=colors, ax=ax)
plt.axhline(mean_mag, color="blue", linestyle="--", label=f"Mean ({mean_mag:.2f})")
plt.title("Magnitude by Tsunami Presence")
plt.xticks([0,1], ["No", "Yes"])
plt.show()

In [ ]:
# Distribution of magnitude according either not or yes thsunami presence
plt.figure(figsize=(10,6))
sns.kdeplot(data=df[df["tsunami"]==0]["magnitude"], fill=True, label="Sin tsunami", color="gray", alpha=0.5)
sns.kdeplot(data=df[df["tsunami"]==1]["magnitude"], fill=True, label="Con tsunami", color="red", alpha=0.5)
plt.axvline(mean_mag, color="blue", linestyle="--", label=f"Media global: {mean_mag:.2f}")
plt.title("Distribución de magnitudes según presencia de tsunami")
plt.xlabel("Magnitud")
plt.legend()
plt.show()

In [ ]:
# Correlations heatmap
from matplotlib.colors import LinearSegmentedColormap
plt.figure(figsize=(10,6))
colors = ["#FFCA00","#FF9D00","#FF7B00","#FF4C00","#D15E00","#FF0000","#FF0084","#FF0055"]
cmap = LinearSegmentedColormap.from_list("custom", colors, N=256)
sns.heatmap(df[["magnitude","depth","gap","dmin","nst","sig"]].corr(), annot=True, cmap=cmap)
plt.title("Correlation Matrix between Earthquake Variables")
plt.show()